<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-02-27 08:44:15--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-02-27 08:44:15--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  4.64MB/s    in 4m 24s  

2024-02-27 08:48:40 (5.25 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [69]:
# method most_simmilar
print(f"Top 5 closest words to 'dog': {wv_embeddings.most_similar(positive='dog', topn=5)}")
print("No, 'cat' not in top 5 for dog.")
print(f"Cat's position is {wv_embeddings.rank('dog', 'cat')}")

Top 5 closest words to 'dog': [('animal', 0.8564180135726929), ('dogs', 0.7880866527557373), ('mammal', 0.7623804211616516), ('cats', 0.7621253728866577), ('animals', 0.760793924331665)]
No, 'cat' not in top 5 for dog.
Cat's position is 26


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [9]:
import numpy as np
import re
import nltk
from nltk.tokenize import WordPunctTokenizer
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text:str):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [49]:
def question_to_vec(question:str, embeddings:list[int], tokenizer, dim:int=200, pre_norm:bool=False, post_norm:bool=False) -> list[int]:
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    tokens = tokenizer().tokenize(question.lower())

    return embeddings.get_mean_vector(keys=tokens, pre_normalize=pre_norm, post_normalize=post_norm, ignore_missing=True)[:dim]

In [12]:
print(question_to_vec('How to use Gensim and NLTK for NLP tasks in Google Colab?', wv_embeddings, WordPunctTokenizer))

[ 0.51679343  0.12172341  0.35998496 -0.4319114  -1.0370318   0.9299659
  1.2020712  -0.36596173  0.44069594  0.15495043 -1.1228906  -0.8447539
 -0.41401604 -0.02108631  1.4079322   0.36768296 -0.42918468  0.30979016
 -0.549481   -0.3993114   0.5301982   1.5252502  -0.95741326 -0.17516652
  0.32587686 -0.7046796  -0.17205478  0.59909433  0.26953658 -0.90959615
 -1.3667134   0.32744414 -1.1641706  -0.8531855   0.7451635   0.44581035
 -0.4981935  -0.66160554  0.98003095  0.46799737 -0.408206   -0.10371072
  1.0035719  -0.28068486  0.16196837  0.08332462 -1.7079794   0.90424365
  0.9915622  -0.09759611  1.2851696   0.18007715 -0.99973404 -1.5195916
 -0.9983058   0.5622769   0.23195772  0.6816965  -0.6364785  -0.2540265
 -0.28495544 -0.1210212  -0.38199216 -0.8660377   0.50349134 -1.6254216
  1.2122718   0.6154348  -0.16097412 -1.1069462   0.9343215  -0.84239435
  0.45441464  0.28014034 -0.6930428  -2.6844242   0.10364325 -0.30110797
  0.091696   -0.61248046 -0.18508728 -0.25780526  0.8865

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [13]:
'''your code'''
tokenizer = WordPunctTokenizer()
tokens = tokenizer.tokenize('I love neural networks')
print(f'Третья компонента равна = {wv_embeddings.get_mean_vector(tokens, pre_normalize=False, ignore_missing=True)[2]:.2f}')

Третья компонента равна = -1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

$ DCG@1 \in [0, 1] $

$ Hits@47 \in [0, 1] $

Однако разность `Hits@47` и `DCG@1` не может быть меньше 0, поскольку у нас не может быть DCG = 1 и Hits = 0. Следовательно

$ Hits@47 - DCG@1 \in [0, 1] $

При $ rank_q $ = 1 мы получаем 0, при $ rank_q \in [2, 47]$ мы получаем 1 (максимум) и при $ rank_q \ge 48 $ мы получаем 0

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

$DCG@10 = \frac{1}{\log_2(1 + 9)} * [9 \le 10] = 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [14]:
def hits_count(dup_ranks: list[int], k:int) -> int:
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    dup_ranks_np = np.array(dup_ranks)
    hits_value = (dup_ranks_np <= k).mean()
    return hits_value

In [15]:
def dcg_score(dup_ranks: list[int], k:int) -> int:
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dup_ranks_np = np.array(dup_ranks)
    activation = dup_ranks_np <= k
    dcg_value = ((1/np.log2(1 + dup_ranks_np)) * activation).mean()

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [16]:
import pandas as pd

In [17]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [18]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!unzip /content/drive/MyDrive/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [21]:
def read_corpus(filename:str) -> list[str]:
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [22]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [23]:
len(validation_data)

3760

Размер нескольких первых строк

In [24]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [50]:
def rank_candidates(question: str, candidates: list[str], embeddings: list[int], tokenizer, dim:int=200, pre_norm: bool = False, post_norm:bool = False):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    sim = cosine_similarity([question_to_vec(question, embeddings, tokenizer, dim=dim, pre_norm=pre_norm, post_norm=post_norm)], [question_to_vec(candidate, embeddings, tokenizer, dim=dim, pre_norm=pre_norm, post_norm=post_norm) for candidate in candidates])[0]
    result = list(zip(sim, range(len(candidates)), candidates))
    result.sort(reverse = True)
    return [res[1:] for res in result]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [28]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [29]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, WordPunctTokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [30]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'),     #скрыт
            (2, 'select2 not displaying search results'),                  #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

ОТВЕТ: `021`

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [31]:
from tqdm.notebook import tqdm

In [32]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, WordPunctTokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [33]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.494 | Hits@   5: 0.578
DCG@  10: 0.516 | Hits@  10: 0.644
DCG@ 100: 0.563 | Hits@ 100: 0.875
DCG@ 500: 0.575 | Hits@ 500: 0.973
DCG@1000: 0.578 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [34]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

### Baseline

In [35]:
words = [tokenizer.tokenize(' '.join(question).lower()) for question in train_data]

In [36]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,              # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,                     # consider words that occured at least 5 times
                 window=5).wv

In [37]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, WordPunctTokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [38]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.287 | Hits@   1: 0.287
DCG@   5: 0.368 | Hits@   5: 0.438
DCG@  10: 0.391 | Hits@  10: 0.510
DCG@ 100: 0.446 | Hits@ 100: 0.780
DCG@ 500: 0.468 | Hits@ 500: 0.952
DCG@1000: 0.473 | Hits@1000: 1.000


## Tokenizers

### ToktokTokenizer

In [39]:
from nltk.tokenize import ToktokTokenizer

tokenizer = ToktokTokenizer()

In [40]:
words = [tokenizer.tokenize(' '.join(question).lower()) for question in train_data]

In [41]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,               # data for model to train on
                 vector_size=200,                  # embedding vector size
                 min_count=5,                      # consider words that occured at least 5 times
                 window=5).wv

In [42]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, ToktokTokenizer, 200)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [43]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.277 | Hits@   1: 0.277
DCG@   5: 0.356 | Hits@   5: 0.427
DCG@  10: 0.378 | Hits@  10: 0.496
DCG@ 100: 0.428 | Hits@ 100: 0.742
DCG@ 500: 0.453 | Hits@ 500: 0.935
DCG@1000: 0.460 | Hits@1000: 1.000


### TREE

In [44]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

In [45]:
words = [tokenizer.tokenize(' '.join(question).lower()) for question in train_data]

In [46]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,               # data for model to train on
                 vector_size=200,                  # embedding vector size
                 min_count=5,                      # consider words that occured at least 5 times
                 window=5).wv

In [47]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, TreebankWordTokenizer, 200)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [48]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.282 | Hits@   1: 0.282
DCG@   5: 0.362 | Hits@   5: 0.432
DCG@  10: 0.380 | Hits@  10: 0.487
DCG@ 100: 0.435 | Hits@ 100: 0.753
DCG@ 500: 0.459 | Hits@ 500: 0.942
DCG@1000: 0.465 | Hits@1000: 1.000


## Normalization

### Pre_norm

In [52]:
tokenizer = WordPunctTokenizer()

In [53]:
words = [tokenizer.tokenize(' '.join(question).lower()) for question in train_data]

In [54]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,               # data for model to train on
                 vector_size=200,                  # embedding vector size
                 min_count=5,                      # consider words that occured at least 5 times
                 window=5).wv

In [55]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, WordPunctTokenizer, 200, True, False)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [56]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.320 | Hits@   1: 0.320
DCG@   5: 0.404 | Hits@   5: 0.477
DCG@  10: 0.429 | Hits@  10: 0.554
DCG@ 100: 0.478 | Hits@ 100: 0.797
DCG@ 500: 0.499 | Hits@ 500: 0.959
DCG@1000: 0.503 | Hits@1000: 1.000


### Post Norm

In [57]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, WordPunctTokenizer, 200, False, True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [58]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.289 | Hits@   1: 0.289
DCG@   5: 0.368 | Hits@   5: 0.436
DCG@  10: 0.394 | Hits@  10: 0.516
DCG@ 100: 0.447 | Hits@ 100: 0.777
DCG@ 500: 0.469 | Hits@ 500: 0.949
DCG@1000: 0.474 | Hits@1000: 1.000


### Pre and Post Norm


In [59]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, WordPunctTokenizer, 200, True, True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [60]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.320 | Hits@   1: 0.320
DCG@   5: 0.404 | Hits@   5: 0.477
DCG@  10: 0.429 | Hits@  10: 0.554
DCG@ 100: 0.478 | Hits@ 100: 0.797
DCG@ 500: 0.499 | Hits@ 500: 0.959
DCG@1000: 0.503 | Hits@1000: 1.000


## Best

In [61]:
import string
from gensim.parsing.preprocessing import STOPWORDS

stop_words = STOPWORDS.union(string.punctuation)
tokenizer = WordPunctTokenizer()

In [62]:
words = [[word for word in tokenizer.tokenize(' '.join(sent).lower()) if word not in stop_words] for sent in train_data]

In [63]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,               # data for model to train on
                 vector_size=200,                  # embedding vector size
                 min_count=10,                     # consider words that occured at least 5 times
                 window=25).wv

In [67]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, WordPunctTokenizer, 200, True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [68]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.475 | Hits@   1: 0.475
DCG@   5: 0.576 | Hits@   5: 0.664
DCG@  10: 0.596 | Hits@  10: 0.727
DCG@ 100: 0.635 | Hits@ 100: 0.915
DCG@ 500: 0.644 | Hits@ 500: 0.984
DCG@1000: 0.646 | Hits@1000: 1.000


### Вывод:
- С baseline лучше всего из протестированных справился WordPunctTokenizer. Возможно причина лежит в том, что такой простой способ токенизации на нашем датасете не плодит лишние слова, что позволяет уместить больше сути в окна, тем самым повысив точность модели по нашим метрикам.
- Пре нормализация улучшает наши результаты значительно, пост нормализация немного тоже помогает, но использовать оба смысла нет, так как тогда результат будет по метрикам как от пре нормализации.
- Если брать только Baseline, то лучше справляются предобученные из Gensim. Скорее всего дело в большом количестве данных для обучения. Если брать лучший результат, то лучше наше решение, потому что оно больше подогнано под нашу задачу.
- Маленькое окно захватывало слишком мало контекста + стоп слова мешали захвату контекста, потому что занимали место в окне.
- Убрать стоп-слова, сделать окно в районе 25 и min_count в районе 10, сделать пре-нормализацию векторов. Изменять длину векторов особо не помогает.

Some text

---



---


In several lines

Cause of colab problems

With PDF

So I

Added some lines

For normal PDF

In [70]:
print('New Line')

Here


In [ ]:
print("New Line")